In [2]:
import json
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [5]:
with open('documents.json', 'rt') as file_input:
    docs_raw = json.load(file_input)

documents=[]

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc['course'] = course_dict['course']
        documents.append(doc)
        

In [7]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [8]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:20<00:00, 45.44it/s]


In [35]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        
    return result_docs

In [36]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    use only the facts from the CONTEXT when answering the QUESTION.
   
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    
    context = ""
    
    for doc in elastic_search(query):
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [37]:
client = MistralClient()

In [38]:
def llm(prompt):
    chat_response = client.chat(
    model="mistral-large-latest",
    messages=[ChatMessage(role="user", content=prompt)]
    )

    return chat_response.choices[0].message.content

In [39]:
query = "How do i execute a command in a running docker container"

In [40]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [41]:
rag(query)

"To execute a command in a running Docker container, you can use the `docker run` command with the appropriate flags and parameters. Here's an example based on the context provided:\n\n```bash\ndocker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1 pgcli -h pg-database -U root -p 5432 -d ny_taxi\n```\n\nIn this command:\n\n- `docker run` is the command to run a command in a new container.\n- `-it` is used to open an interactive terminal.\n- `--rm` is used to automatically remove the container when it exits.\n- `--network pg-network` specifies the network to connect to.\n- `ai2ys/dockerized-pgcli:4.0.1` is the Docker image to use.\n- `pgcli -h pg-database -U root -p 5432 -d ny_taxi` is the command to run inside the container. Here, `pgcli` is the command-line interface for PostgreSQL, `-h pg-database` specifies the hostname, `-U root` specifies the username, `-p 5432` specifies the port, and `-d ny_taxi` specifies the database name."